# Coding Temple's Data Analytics Program
---
## Python for DA: Weekend Project

For this weekend project, you will be connecting to the [Disney API](https://disneyapi.dev/) to create an ETL pipeline. Your project should contain:

- etl_pipeline.py
    - Loads in data from the API object for all characters
    - Stores required fields from the API to a DataFrame
        - name
        - all movies/shows the character appeared in
        - any allies
        - any enemies
        - any park attractions
    - Cleans the data
    - Performs any transformations/feature engineering you wish to complete
    - Stores the data in an ElephantSQL server
    - Stores the data in a .csv file

- notebook.ipynb
    - Contains all cells you used to test your code before loading it into the pipeline
    - Loads in the data from your .csv file
    - Conduct EDA through data
    - Conduct an analysis on your dataset!

In [1]:
import requests
import pandas as pd
from PIL import Image
from io import BytesIO

Testing my fields to get the right fields.  

In [15]:
#I would likelt change this to be more dynamic of a url to get count in the future
#or set the number to 1000000 since it auto changes it to the max count of characters. 
#but that also depends on performance.
url = f'https://api.disneyapi.dev/character?page=1&pageSize=8000'
response= requests.get(url)
#creating a counting variable
max_cnt=response.json()['info']['count']
#create a variable to shorthand the data we will be indexing through
my_data = response.json()['data']

#columns list for PANDA PANDA PANDA PANDA
c=['name','appearances','allies','enemies','park_attractions','appearance_cnt','ally_cnt','enemy_cnt','park_attractions_cnt','image_blob']



print(max_cnt)



7438


In [ ]:




# fairly straightforward here just grabbing some pokemon and stats
def catchpoke(name):
    url = f'https://api.disneyapi.dev/character?page=1&pageSize=8000'
    response= requests.get(url)
    my_data = response
    my_data.json()
    poke_dict = {
        'name' : name,
        'ability 1' : my_data.json()['abilities'][0]['ability']['name'],
        'base_experience' : my_data.json()['base_experience'],
        'front_shiny_png' : my_data.json()['sprites']['front_shiny'],
        'attack_stat_base' : my_data.json()['stats'][1]['base_stat'], 
        'hp_stat_base' : my_data.json()['stats'][0]['base_stat'],
        'defense_stat_base' : my_data.json()['stats'][2]['base_stat']}    
    
    #block for image
    img_str = my_data.json()['sprites']['front_shiny']
    response = requests.get(img_str)
    img= Image.open(BytesIO(response.content))
    img =img.convert('RGB')
    output_image_path = 'output_image.jpg'
    img.save(output_image_path)
    #display image
    display(img)

    #return dictionary
    return '\n'.join(f'{keys} : {val}' for keys,val in poke_dict.items())